In [1]:
import numpy as np
import matplotlib as plt
import random

In [2]:
# Define the environment

def get_env():
    '''
    Arguments: 
            None
    Returns:
            states: array of states in the gridworld
            goal_state: tuple containing the goal state
            start_state: tuple containing the start state
            wind: array containing the wind strengths of different columns
            num_rows: number of rows in the gridworld
            num_columns: number of columns in the gridworld
    '''
    num_rows = 7
    num_columns = 10
    goal_state = (3,7)
    start_state = (3,0)
    num_states = num_rows*num_columns
    num_actions = 4
    wind = [0,0,0,1,1,1,2,2,1,0]
    states = []
    for i in range(num_rows):
        for j in range(num_columns):
            states.append((i,j))
            
    return states,goal_state,start_state,wind,num_rows,num_columns

In [3]:
# Make the necessary move 

def make_move(current_position,move,goal_state,wind):
    '''
    Arguments: 
            current_position: tuple containing the current position of the agent
            move: 'l','r','u' or 'd'
            goal_state: tuple containing the final state
            wind: array containing the wind strengths of different columns
    Returns:
            new_position: tuple containing the new position of the agent after the move is made
            reward: reward assigned to the move made at the state depending on the next state reached
    '''
    
    # Convert tuple to list so the operations are easier
    current_position = list(current_position)
    new_position = current_position
    
    # While making a move, it must be ensured that the agent doesn't go out of the environment
    if move == 'l':
        new_position[1] = max(0,current_position[1] - 1)
    elif move == 'r':
        new_position[1] = min(9,current_position[1] + 1)
    elif move == 'u':
        new_position[0] = max(0,current_position[0] - 1)
    elif move == 'd':
        new_position[0] = min(6,current_position[0] + 1)
    
    # Account for the wind in the corresponding column
    new_position[0] -= wind[new_position[1]]
    if new_position[0] < 0:
        new_position[0] = 0
    
    # Reward is 1 if new position is the goal state and -1 otherwise
    reward = 0
    new_position = tuple(new_position)
    if new_position == goal_state:
        reward = 1
    else:
        reward = -1
    
    return new_position,reward

In [4]:
# Find the best action

def best_action(Q,state):
    '''
    Arguments:
            Q: The state-action value function
            state: current state of the agent
    Returns:
            max_action: The optimal action in the state
    '''
    val = -1e17
    max_action = 'l'
    # Iterate through all state action-pairs for the state and find the best one
    for action,value in Q[state].items():
        if value > val:
            val = value
            max_action = action
    
    return max_action


# Find an action using epsilon-greedy policy

def get_action(Q,state,epsilon):
    '''
    Arguments:
            Q: The state-action value function
            state: current state of the agent
    Returns:
            action: action picked by the epsilon-greedy policy
    '''
    val = np.random.rand()
    action = ''
    # if val > epsilon, choose the best action, else choose a random action
    if val > epsilon:
        action = best_action(Q,state)
    else:
        list_actions = get_possible_actions(state)
        action = random.choice(list_actions)

    return action

In [5]:
# Find the list of possible action in a state

def get_possible_actions(state):
    '''
    Arguments:
            state: one of the states of the gridworld
    Returns:
            actions: list of possible actions from the given state
    '''
    actions = []
    if state[0] > 0:
        actions.append('u')
    if state[0] < 6:
        actions.append('d')
    if state[1] > 0:
        actions.append('l')
    if state[1] < 9:
        actions.append('r')
    
    return actions

In [6]:
# Implementation of sarsa

def sarsa(states,goal_state,start_state,wind,alpha = 0.1,epsilon = 0.1,gamma = 1,episodes=100):
    '''
    Arguments:
            states: array of states in the gridworld
            goal_state: tuple containing the goal state
            start_state: tuple containing the start state
            wind: array containing the wind strengths of different columns
            alpha: step-size parameter
            gamma: discount factor(1 here as the task is undiscounted)
            episodes: number of episodes for training
    Returns:
            Q: state-action value function after sarsa is finished
    '''
    # Initialize the state-action value function
    Q = {}
    for state in states:
        Q[state] = {}
        possible_actions = get_possible_actions(state)
        for action in possible_actions:
            Q[state][action] = 0

    # Iterate for required number of episodes
    for episode in range(1,episodes+1):
        # Initialize current state and action
        current_state = start_state
        current_action = get_action(Q,current_state,epsilon/episode)
        
        # Loop until goal is not reached
        while current_state != goal_state:
            
            next_state,reward = make_move(current_state,current_action,goal_state,wind)
            next_action = get_action(Q,next_state,epsilon/episode)       # Find next action based on epsilon-greedy policy

            # Sarsa update
            Q[current_state][current_action] = Q[current_state][current_action] + alpha*(reward + gamma*Q[next_state][next_action] - Q[current_state][current_action])
            
            current_state = next_state
            current_action = next_action
            if current_state == goal_state:
                break
    return Q

In [7]:
# Optimal policy 

def optimal_policy(Q,states,goal_state,start_state,wind,num_rows,num_columns):
    '''
    Arguments:
            Q: state-action value function after sarsa is completed
            states: array of states in the gridworld
            goal_state: tuple containing the goal state
            start_state: tuple containing the start state
            wind: array containing the wind strengths of different columns
            num_rows: number of rows in the gridworld
            num_columns: number of columns in the gridworld
    Returns:
            visiting: array containing the steps taken in optimal path
            returns: return when optimal path is followed
            path: array containing the optimal path
            
    '''
    current_state = start_state
    path = []
    path.append(current_state)
    visiting = np.zeros((num_rows,num_columns))
    steps = 1
    returns = 0
    # Follow greedy policy from start state until goal is reached
    while 1:
        action = best_action(Q,current_state)
        next_state,r = make_move(current_state,action,goal_state,wind)
        returns += r
        visiting[next_state[0]][next_state[1]] = steps
        current_state = next_state
        steps += 1
        path.append(current_state)
        if current_state == goal_state:
            break
            
    return visiting,returns,path

In [8]:
def windy_gridworld():

    # Define env
    states,goal_state,start_state,wind,num_rows,num_columns = get_env()
    
    # Find state action value function
    Q = sarsa(states,goal_state,start_state,wind,alpha = 0.5,epsilon = 0.1,gamma = 1,episodes = 1000)
    
    # Find optimal path
    visiting,reward,path = optimal_policy(Q,states,goal_state,start_state,wind,num_rows,num_columns)
    
    print("The optimal path is:")
    path = '->'.join([str(item) for item in path])
    print(path,'\n')
    print('The path taken can be visualized as:')
    for i in range(num_rows):
        for j in range(num_columns):
            print('{:1.0f}'.format(visiting[i][j]), end="\t")
        print()
    print(f'\nThe optimal return is {reward}')

In [9]:
windy_gridworld()

The optimal path is:
(3, 0)->(3, 1)->(3, 2)->(2, 3)->(1, 4)->(0, 5)->(0, 6)->(0, 7)->(0, 8)->(0, 9)->(1, 9)->(2, 9)->(3, 9)->(4, 9)->(5, 9)->(6, 9)->(5, 8)->(3, 7) 

The path taken can be visualized as:
0	0	0	0	0	5	6	7	8	9	
0	0	0	0	4	0	0	0	0	10	
0	0	0	3	0	0	0	0	0	11	
0	1	2	0	0	0	0	17	0	12	
0	0	0	0	0	0	0	0	0	13	
0	0	0	0	0	0	0	0	16	14	
0	0	0	0	0	0	0	0	0	15	

The optimal return is -15
